# Converters

Converters are used to transform prompts before sending them to the target.

This can be useful for a variety of reasons, such as encoding the prompt in a different format, or adding additional information to the prompt. For example, you might want to convert a prompt to base64 before sending it to the target, or add a prefix to the prompt to indicate that it is a question.

## Simple Converter Example

Converters can be used to perform these types of transformations. Here is a simple program that uses Rot13Converter converter, RandomCapitalLettersConverter, and AsciiArtConverter.

In [1]:

from pyrit.prompt_converter import (
    AsciiArtConverter,
    BinaryConverter,
    FirstLetterConverter,
    RandomCapitalLettersConverter,
    ROT13Converter,
)
from pyrit.setup import IN_MEMORY, initialize_pyrit_async

await initialize_pyrit_async(memory_db_type=IN_MEMORY)  # type: ignore

prompt = "tell me how to cut down a tree"


print(await ROT13Converter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await RandomCapitalLettersConverter(percentage=25.0).convert_tokens_async(prompt=prompt))  # type: ignore
print(await AsciiArtConverter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await BinaryConverter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await FirstLetterConverter().convert_tokens_async(prompt=prompt))  # type: ignore

Found default environment files: ['C:\\Users\\rlundeen\\.pyrit\\.env', 'C:\\Users\\rlundeen\\.pyrit\\.env.local']
Loaded environment file: C:\Users\rlundeen\.pyrit\.env
Loaded environment file: C:\Users\rlundeen\.pyrit\.env.local
text: gryy zr ubj gb phg qbja n gerr
text: Tell me How to Cut Down a treE
text:   /^^             /^^ /^^                                                          /^^                              /^^        /^^                                             /^^                             
  /^^             /^^ /^^                         /^^                              /^^                              /^^        /^^                                             /^^                             
/^/^ /^   /^^     /^^ /^^ /^^^ /^^ /^^    /^^     /^^        /^^    /^^     /^^^ /^/^ /^   /^^        /^^^/^^  /^^/^/^ /^      /^^   /^^    /^^     /^^^/^^ /^^      /^^     /^/^ /^/^ /^^^   /^^       /^^    
  /^^   /^   /^^  /^^ /^^  /^^  /^  /^^ /^   /^^  /^ /^    /^^  /^